In [10]:
import os
import weaviate
import tiktoken
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import StorageContext, VectorStoreIndex, get_response_synthesizer, Settings
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core import PromptTemplate

PROMPT_TEMPLATE = """Du bist ein hilfreicher deutscher Assistent für junge Eltern, der Informationen aus Foren sammelt, um Eltern bei der Beantwortung von Fragen über ihre Kinder zu helfen.

            Verwende die folgenden Konversationen aus Elternforen, um den Nutzer über ein bestimmtes Thema zu helfen. Jede Zeile beginnt mit dem Namen des Benutzers, gefolgt von ":" und dann dem Kommentar, zum Beispiel so: "John: Bei mir ist es genauso."
            Verschiedene Konversationen können sich auf dasselbe Thema beziehen.
            <conversations>
            {context_str}
            </conversations>
            
            Wenn du die Unterhaltungen im Forum zitierst, gib bitte den Benutzernamen in deiner Antwort an. Hier sind Beispielw in <example> Tags:
            <example>
            Viele Nutzer sagen dass es normal ist Kinder nachts zu stillen. Cari234 sagt, z.B, dass sie täglich überfordert ist. Lomo2 hat gleiche Erfahrungen.
            </example>
            <example>
            Die Konversationen beinhalten keine Hinweise ob es normal ist, dass Kinder nachts Wachfenster haben.
            </example>

            Hier ist deine Aufgabe: Welche relevanten Informationen kannst du aus den oben genannten Gesprächen zu diesem Thema entnehmen? Das Thema ist unten in <question>-Tags:
            <question>
            {query_str}
            </question>

            Um deine Aufgabe zu erledigen, gehe die folgenden Schritte durch:
            1. Erstelle eine umfassende Zusammenfassung für jede Konversation in <conversation> Tags oben. Die Zusammenfassungen sollte alle wichtigen Punkte und Hauptgedanken des Originaltextes die sich auf diesem Aufgabe Thema beziehen abdecken und gleichzeitig die Informationen in einem prägnanten und leicht verständlichen Format zusammenfassen.
            Achte bitte darauf, dass die Zusammenfassung die Usernamen, relevante Details und Beispiele enthält, die die Hauptgedanken unterstützen, und vermeide unnötige Informationen oder Wiederholungen.
            2. Erledige deine Aufgabe auf der Grundlage der Zusammenfassungen von Schritt 1. Füge alle relevanten Informationen, Details und Beispiele ein die aus der Zusammenfassungen rauskommen und sich auf diesem Thema beziehen. Behalte die Antwort auf maximal 5 Sätze.

            Wenn du die Aufgabe erledigst, nenn bitte konkrete Beispiele und Tipps aus den Forendiskussionen und verallgemeinere die Details nicht. Wenn du die Antwort nicht weißt, sag einfach, dass du es nicht weißt.
            Fang deine Antwort mit "Das sagen andere Nutzer dazu:" an. Danach, gib die Zusammenfassungen von Schritt 1 aus als Bulletpoint-Liste aus. Für jede Konversation soll es ein Bulletpoint geben. Gib dann deine zusammenfassende Antwort gemäß Schritt 2 in eine neue Zeile ein.
            Hier sind Beispiele wie deine Antworten formattiert werden sollen, in <answer-example>-Tags:

            <answer-example>
            Das sagen andere Nutzer dazu:
            - jomda erklärt, dass Babys manchmal schreien, wenn ältere Geschwister versorgt werden müssen oder wenn das Baby nachts wach wird, weil der Betreuer kurz etwas erledigen muss.
            - Mami83 berichtet, dass ihr Baby auch Phasen hatte, in denen es sehr unruhig war und häufig nachts aufwachte. Caro34 bestätigt.

            Das sagen die Nutzer zusammengefasst: Häufiges nächtliches Aufwachen und Unruhe sind bei Babys oft normal und hängen mit deren Entwicklung und Bedürfnissen zusammen. Die Situation kann vorübergehend sehr herausfordernd sein, aber die Eltern müssen durchhalten, da es mit der Zeit wieder besser wird.
            </answer-example>
            <answer-example>
            Das sagen andere Nutzer dazu:
            - bilbo45 sagt, dass solche schwierigen Phasen normal sind und immer wieder kommen können, da sich Babys ständig weiterentwickeln. Sie ermutigt, durchzuhalten, da es mit der Zeit besser wird.
            - Micebwn beschreibt, dass ihr 9 Monate altes Baby seit Tagen nachts weinend aufwacht.

            Aus den Gesprächen geht hervor, dass bei den meisten Kindern dieser spezielle Test mit schwarzen und weißen Punkten, bei denen das Kind etwas Bestimmtes erkennen und zeigen soll, im Alter von 11 Monaten noch nicht funktioniert. Die Eltern berichten, dass ihre Kinder stattdessen eher versuchen, mit den Fingern in den Mund der Ärzte zu kommen oder generell eher an der Untersuchung interessiert sind als an dem Test.
            </answer-example>    

            Erledige jetzt bitte deine Aufgabe bezüglich des Themas von oben.
            """
EMBEDDING_MODEL = 'aari1995/German_Semantic_STS_V2'
# MAX_TOKENS = 4000

groq_api_key = os.environ.get("GROQ_API_KEY")

llm =  Groq(
            model="llama3-70b-8192",
            api_key=groq_api_key,
            # max_tokens=MAX_TOKENS
        )
# llm =  Groq(
#             model="mixtral-8x7b-32768",
#             api_key=groq_api_key,
#         )
# llm = Anthropic(model="claude-3-haiku-20240307")
embedding_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
# Settings.tokenizer = tokenizer
Settings.llm = llm
Settings.embed_model = embedding_model
# token_counter = TokenCountingHandler(
#     tokenizer=tiktoken.encoding_for_model("llama3-70b-8192").encode
# )
# Settings.callback_manager = CallbackManager([token_counter])

client = weaviate.Client("http://localhost:8080")
vector_store = WeaviateVectorStore(weaviate_client = client, embed_model=embedding_model, index_name="BabyForum")
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    # filters=filters,
)
response_synthesizer = get_response_synthesizer(llm = llm, response_mode="simple_summarize")
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.75)],
)

qa_prompt_tmpl = PromptTemplate(PROMPT_TEMPLATE)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

No sentence-transformers model found with name aari1995/German_Semantic_STS_V2. Creating a new one with MEAN pooling.


In [11]:
answer1 = query_engine.query("Kind hat ein Stein verschluckt")

ValueError: Calculated available context size -1200 was not non-negative.

In [ ]:
count = 0
for source in answer1.source_nodes:
    print(f"Source #{ count } has score {source.score}:\n{source.text[:100]}")
    count += 1

print(answer1)

Source #0 has score 0.80401653:
lizzywithD: danke dir! das mit dem im eigenen zimmer schlafen überlege ich mir ständig aber ich kann
Source #1 has score 0.79711473:
SteMatz: Schlichtweg ... JA. Meine Maus schläft auch durch, leider sehr unruhig und ich werde bei je
Source #2 has score 0.79370987:
Mami823: War bei uns genauso. Als er ca. 3,5 Monate alt war, habe ich mein Baby nicht wiedererkannt.
Source #3 has score 0.79095107:
Sonnenschein729: Hallo 🙋🏼‍♀️ mein Sohn wird nächste Woche erst fünf Monate alt. Liest sich als wäre 
Source #4 has score 0.79071355:
Thema:ist das noch normal?
lizzywithD: hallo community
beitrag nr. 4 in kürzerer zeit. 
thema: babys
Source #5 has score 0.78898525:
Thema:Baby macht in der Nacht Situps
LiNo33: Hallo zusammen,
mein kleiner schläft seit einigen Woche
Source #6 has score 0.78773314:
Thema:Baby 21 Wochen alt
Vanni9: Hallo zusammen, 
Unser kleine ist heute 21 Wochen alt. Seit circa 2
Source #7 has score 0.7872818699999999:
Tanja.SoSo: Hallöchen und ein